# Hands-on 1: QueryFusionRetriever

### Problem
Given the Paul Graham’s essay build an intelligent QueryFusionRetriever that synthesizes VectorStoreIndex and BM25Retriever to precisely extract the most relevant contextual nodes to answer the question:​

"Why was the author in Florence?"

### Suggested tasks:
- 🔤 Create & validate a VectorStoreIndex​
- 🔤 Create & validate a BM25Retriever​
- 🚀 Create a QueryFusionRetriever merging vector and BM25 retrieval strategies​
- 📊 Compare Retrieval Strategies

## Code

In [ ]:
# if running on colab uncomment the those lines
%pip install httpx==0.27.2
%pip install llama-index==0.12.3
%pip install llama-index-retrievers-bm25>=0.50
%pip install openai==1.57.0
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
# if you want to load .env uncomment the following lines otherwise you can set the environment variables directly
# from dotenv import load_dotenv
# load_dotenv()

import os
os.environ["OPENAI_API_KEY"] = "sk-" # set your openai api key here

True

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.base.base_retriever import BaseRetriever
from rich import print as rprint
from llama_index.core.schema import MetadataMode
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.retrievers.fusion_retriever import FUSION_MODES

resource module not available on Windows


c:\Users\n.fretti\Desktop\projects\rag_and_roll\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
documents = SimpleDirectoryReader("./data/paul_graham").load_data()
splitter = SentenceSplitter(chunk_size=256)

index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter], show_progress=True
)

Generating embeddings: 100%|██████████| 587/587 [00:09<00:00, 58.79it/s]


In [5]:
QUESTION = "Why the author was in Florance?"

### Vector retrieval

In [6]:
def init_vector_retriever(index: VectorStoreIndex, similarity_top_k:int) -> BaseRetriever:
    retriever = index.as_retriever(similarity_top_k=similarity_top_k)
    return retriever

In [12]:
vector_retriever = init_vector_retriever(index, similarity_top_k=3)
vector_nodes = vector_retriever.retrieve(QUESTION)

for node in vector_nodes:
    rprint(node.get_content(
        metadata_mode=MetadataMode.LLM
    ))

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

I had some money saved from consulting work I'd done in grad school; there was probably enough to last a year if I 
lived cheaply. Now all I had to do was learn Italian.

Only stranieri (foreigners) had to take this entrance exam. In retrospect it may well have been a way of excluding 
them, because there were so many stranieri attracted by the idea of studying art in Florence that the Italian 
students would otherwise have been outnumbered. I was in decent shape at painting and drawing from the RISD 
foundation that summer, but I still don't know how I managed to pass the written exam. I remember that I answered 
the essay question by writing about Cezanne, and that I cranked up the intellectual level as high as I could to 
make the most of my limited vocabulary. [2]

I'm only up to age 25 and already there are such conspicuous patterns. Here I was, yet again about to attend some 
august institution in the hopes of learning about some prestigious subject, and yet again about to be disappointed.

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

This was now only weeks away. My nice landlady let me leave my stuff in her attic. I had some money saved from 
consulting work I'd done in grad school; there was probably enough to last a year if I lived cheaply. Now all I had
to do was learn Italian.

Only stranieri (foreigners) had to take this entrance exam. In retrospect it may well have been a way of excluding 
them, because there were so many stranieri attracted by the idea of studying art in Florence that the Italian 
students would otherwise have been outnumbered. I was in decent shape at painting and drawing from the RISD 
foundation that summer, but I still don't know how I managed to pass the written exam. I remember that I answered 
the essay question by writing about Cezanne, and that I cranked up the intellectual level as high as I could to 
make the most of my limited vocabulary. [2]

I'm only up to age 25 and already there are such conspicuous patterns.

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

So in 1993 I dropped out. I hung around Providence for a bit, and then my college friend Nancy Parmet did me a big 
favor. A rent-controlled apartment in a building her mother owned in New York was becoming vacant. Did I want it? 
It wasn't much more than my current place, and New York was supposed to be where the artists were. So yes, I wanted
it! [7]

Asterix comics begin by zooming in on a tiny corner of Roman Gaul that turns out not to be controlled by the 
Romans. You can do something similar on a map of New York City: if you zoom in on the Upper East Side, there's a 
tiny corner that's not rich, or at least wasn't in 1993. It's called Yorkville, and that was my new home. Now I was
a New York artist — in the strictly technical sense of making paintings and living in New York.

I was nervous about money, because I could sense that Interleaf was on the way down.

### Setup bm25 retriever

In [13]:
def init_base25_retriever(index: VectorStoreIndex, similarity_top_k:int) -> BM25Retriever:
    retriever = BM25Retriever.from_defaults(
        docstore=index.docstore, similarity_top_k=similarity_top_k
    )
    return retriever

In [14]:

bm25_retriever = init_base25_retriever(index, similarity_top_k=3)
bm25_nodes = bm25_retriever.retrieve(QUESTION)
for node in bm25_nodes:
    rprint(node.get_content(
        metadata_mode=MetadataMode.LLM
    ))

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

That's not always why artists have a signature style, but it's usually why buyers pay a lot for such work. [6]

There were plenty of earnest students too: kids who "could draw" in high school, and now had come to what was 
supposed to be the best art school in the country, to learn to draw even better. They tended to be confused and 
demoralized by what they found at RISD, but they kept going, because painting was what they did. I was not one of 
the kids who could draw in high school, but at RISD I was definitely closer to their tribe than the tribe of 
signature style seekers.

I learned a lot in the color class I took at RISD, but otherwise I was basically teaching myself to paint, and I 
could do that for free. So in 1993 I dropped out. I hung around Providence for a bit, and then my college friend 
Nancy Parmet did me a big favor.

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

At least not the painting department. The textile department, which my next door neighbor belonged to, seemed to be
pretty rigorous. No doubt illustration and architecture were too. But painting was post-rigorous. Painting students
were supposed to express themselves, which to the more worldly ones meant to try to cook up some sort of 
distinctive signature style.

A signature style is the visual equivalent of what in show business is known as a "schtick": something that 
immediately identifies the work as yours and no one else's. For example, when you see a painting that looks like a 
certain kind of cartoon, you know it's by Roy Lichtenstein. So if you see a big painting of this type hanging in 
the apartment of a hedge fund manager, you know he paid millions of dollars for it. That's not always why artists 
have a signature style, but it's usually why buyers pay a lot for such work.

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

It was exciting for a while. Painting started to go better. I experimented with a new kind of still life where I'd 
paint one painting in the old way, then photograph it and print it, blown up, on canvas, and then use that as the 
underpainting for a second still life, painted from the same objects (which hopefully hadn't rotted yet).

Meanwhile I looked for an apartment to buy. Now I could actually choose what neighborhood to live in. Where, I 
asked myself and various real estate agents, is the Cambridge of New York? Aided by occasional visits to actual 
Cambridge, I gradually realized there wasn't one. Huh.

Around this time, in the spring of 2000, I had an idea. It was clear from our experience with Viaweb that web apps 
were the future. Why not build a web app for making web apps? Why not let people edit code on our server through 
the browser, and then host the resulting applications for them?

### Merge the results of the two retrievers

In [10]:
def init_query_fusion_retriever(retrievers: list[BaseRetriever], similarity_top_k:int, mode: FUSION_MODES, num_queries:int = 1) -> QueryFusionRetriever:
    retriever = QueryFusionRetriever(
        retrievers=retrievers,
        similarity_top_k=similarity_top_k,
        num_queries=num_queries,  # set this to 1 to disable query generation
        mode=mode,
        use_async=True,
        verbose=True,
        # query_gen_prompt="...",  # we could override the query generation prompt here
    )
    return retriever

In [15]:
retriever = init_query_fusion_retriever([vector_retriever, bm25_retriever], similarity_top_k=3, mode=FUSION_MODES.RECIPROCAL_RANK)
nodes = retriever.retrieve(QUESTION)
for node in nodes:
    rprint(node.get_content(
        metadata_mode=MetadataMode.LLM
    ))

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

I had some money saved from consulting work I'd done in grad school; there was probably enough to last a year if I 
lived cheaply. Now all I had to do was learn Italian.

Only stranieri (foreigners) had to take this entrance exam. In retrospect it may well have been a way of excluding 
them, because there were so many stranieri attracted by the idea of studying art in Florence that the Italian 
students would otherwise have been outnumbered. I was in decent shape at painting and drawing from the RISD 
foundation that summer, but I still don't know how I managed to pass the written exam. I remember that I answered 
the essay question by writing about Cezanne, and that I cranked up the intellectual level as high as I could to 
make the most of my limited vocabulary. [2]

I'm only up to age 25 and already there are such conspicuous patterns. Here I was, yet again about to attend some 
august institution in the hopes of learning about some prestigious subject, and yet again about to be disappointed.

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

That's not always why artists have a signature style, but it's usually why buyers pay a lot for such work. [6]

There were plenty of earnest students too: kids who "could draw" in high school, and now had come to what was 
supposed to be the best art school in the country, to learn to draw even better. They tended to be confused and 
demoralized by what they found at RISD, but they kept going, because painting was what they did. I was not one of 
the kids who could draw in high school, but at RISD I was definitely closer to their tribe than the tribe of 
signature style seekers.

I learned a lot in the color class I took at RISD, but otherwise I was basically teaching myself to paint, and I 
could do that for free. So in 1993 I dropped out. I hung around Providence for a bit, and then my college friend 
Nancy Parmet did me a big favor.

file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\lesson_02\data\paul_graham\paul_graham_essay.txt

This was now only weeks away. My nice landlady let me leave my stuff in her attic. I had some money saved from 
consulting work I'd done in grad school; there was probably enough to last a year if I lived cheaply. Now all I had
to do was learn Italian.

Only stranieri (foreigners) had to take this entrance exam. In retrospect it may well have been a way of excluding 
them, because there were so many stranieri attracted by the idea of studying art in Florence that the Italian 
students would otherwise have been outnumbered. I was in decent shape at painting and drawing from the RISD 
foundation that summer, but I still don't know how I managed to pass the written exam. I remember that I answered 
the essay question by writing about Cezanne, and that I cranked up the intellectual level as high as I could to 
make the most of my limited vocabulary. [2]

I'm only up to age 25 and already there are such conspicuous patterns.